In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"), 
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("{ingredients} 재료를 사용하여 만들 수 있는 요리 레시피를 추천해주세요.")

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행 및 결과 출력
ingredients = "토마토, 양파, 치즈"
recipe = chain.invoke({"ingredients": ingredients})
print(recipe)

토마토·양파·치즈만 있으면 15분 만에 완성하는 “팬 하나” 요리  
**“치즈 토마토 양파 컬럼버스”** (Columbus = 구운 빵 위에 올려 먹는다는 뜻)

---

### 1. 재료 (1인 기준)
- 토마토 1개(150 g)  
- 양파 ½개(100 g)  
- 가열되기 쉬운 치즈 60 g(모짜렐라·피자·슈레드 뭐든 OK)  
- 올리브오일 1T(또는 식용유)  
- 소금, 후추 약간  
- (선택) 마른 오레가노 or 바질 한 꼬집  
- (선택) 바게트·식빵·크래커

---

### 2. 준비
1. 토마토는 1 cm 두께로 썰고, 양파는 0.5 cm 두께로 채 썬다.  
2. 치즈는 한입 크기로 찢어 두면 녹을 때 더 멋진 스트링이 생긴다.

---

### 3. 조리 (전체 10~12분)
1. **팬 달궜다가 중불로 줄이기**  
   팬에 올리브오일 1T를 두르고 센 불에서 30초 달군 뒤 중불로 줄인다.  
2. **양파 먼저**  
   양파를 넣고 소금 한 꼬집, 후추 두 바퀴. 3분간 자주 저어가며 투명+자글자글하게.  
3. **토마토 투하**  
   토마토를 넣고 1분간 빠르게 볶다가, 이때 토마토에서 나온 물이 증발하면서 자연스럽게 소스가 된다.  
4. **치즈 토핑**  
   불을 끄고 바로 치즈를 고르게 뿌린 뒤 뚜껑을 덮거나 알루미늄 호일로 덮어 잔열로 2~3분간 녹인다.  
   *(치즈가 끓는 김에 토마토 양파와 뒤섞이며 크림 같은 소스가 된다.)*  
5. **마무리**  
   오레가노 한 꼬집 뿌리고, 뜨거울 때 바로 그릇에 담는다.

---

### 4. 먹는 법
- **그냥 숟가락으로** → 저탄수·키토 간식  
- **빵 위에 올려서** → 브루스케타 스타일  
- **삶은 파스타에 버무려** → 10초만에 로제 파스타 완성  
- **계란 프라이를 올려** → 해장 아침식사

---

### 5. 꿀팁
- 토마토가 물컹하면 1t의 토마토케첩을 넣으면 색·맛 모두 업그레이드.  
- 치즈가 없을 땐 1T 우

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
from dotenv import load_dotenv

load_dotenv()

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"), 
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

# 1단계 프롬프트 및 체인
prompt1 = ChatPromptTemplate.from_template("{destination}의 대표적인 관광 명소 한 가지를 추천해주세요.")
chain1 = prompt1 | llm | StrOutputParser()

# 2단계 프롬프트 및 체인
prompt2 = ChatPromptTemplate.from_template("{attraction}에 대한 상세한 정보를 알려주세요. (역사, 특징, 방문 팁 등)")
chain2 = prompt2 | llm | StrOutputParser()

# 두 체인 연결
# RunnablePassthrough를 사용하여 1단계 출력을 2단계 입력으로 전달
# itemgetter를 사용하지 않고도 동일한 기능 구현
combined_chain = {"attraction": chain1} | RunnablePassthrough.assign(details=chain2)


# 체인 실행 및 결과 출력
destination = "로마"
result = combined_chain.invoke({"destination": destination})

print(f"입력: \"{destination}\"")
print(f"1단계 결과: \"{result['attraction']}\"")
print(f"2단계 결과: 명소: {result['attraction']}\n{result['details']}")

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"), 
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

# 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "대한민국 축구 국가대표팀이 월드컵 예선에서 강력한 상대를 만나 승리하며 본선 진출에 한 걸음 더 다가섰다. 손흥민 선수의 활약이 돋보였다.",
        "keywords": "축구, 월드컵, 손흥민"
    }
]

# 예시 프롬프트
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사에서 핵심 키워드 3개를 추출하는 전문가입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

# 체인 생성
chain = final_prompt | llm | StrOutputParser()

# 테스트 뉴스
test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 체인 실행 및 결과 출력
keywords = chain.invoke({"input": test_news})
print(keywords)